In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn

from transformers import BertTokenizer, BertModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split

import pickle

c:\Users\Kyriakos\anaconda3\envs\nlp1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# read data 
df = pd.read_csv('dataset\listings_comments.csv')

In [4]:
# load comments embeddings
with open('embeddings\comments_embeddings.pkl', 'rb') as f:
    comments_embeddings = pickle.load(f)

In [5]:
comments_embeddings.shape

(6998, 384)

In [8]:
print(df.review_scores_rating.isna().sum())
# fill with mean
df.review_scores_rating.fillna(df.review_scores_rating.mean(), inplace=True)

print(df.review_scores_rating.isna().sum())

682
0


In [14]:
#  define X and y for training
X = comments_embeddings
y = df['review_scores_rating'].values

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



    

In [13]:
from model.models.regressor import MLPRegressor

from torch.utils.data import TensorDataset, DataLoader
import pytorch_lightning as pl

# Define the PyTorch DataLoader for training and testing data
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the MLPRegressor model
input_dim = X.shape[1] # get the size of the input dimension from the data
hidden_dim = 32
model = MLPRegressor(input_dim, hidden_dim)

# Define the PyTorch Lightning Trainer
trainer = pl.Trainer(gpus=1, max_epochs=10)

# Train the model
trainer.fit(model, train_loader)

# Test the model
trainer.test(model, test_loader)



